# Exercises for Introduction to Python for Data Science

Week 11 – Statistics & Hypothesis Testing

Matthias Feurer and Andreas Bender

In this sheet we will sharpen our statistical tool‑set. We begin with
sampling and descriptive statistics, move on to classical hypothesis
tests (paired, unpaired and goodness‑of‑fit), explore modern bootstrap
and Monte‑Carlo testing, and finish with a small linear‑model analysis
in *statsmodels*.

**Learning goals**

-   Generate random samples from various distributions.
-   Compute higher‑order descriptive statistics (variance, skewness,
    kurtosis).
-   Perform paired/unpaired *t*‑tests and a χ² goodness‑of‑fit test.
-   Build resampling‑based confidence intervals and Monte‑Carlo
    p‑values.
-   Fit and interrogate an ordinary least‑squares (OLS) model.

------------------------------------------------------------------------

## Common imports

Run this once; it is reused in all subsequent exercises.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from statsmodels.formula.api import ols
plt.close('all')  # fresh state

## Exercise 1 – *Sampling & higher‑order moments*

Draw **2000** random numbers from two distributions

-   `normal_sample` – standard normal $\mathcal{N}(0,1)$  
-   `laplace_sample` – Laplace distribution centred at 0 with scale
    $1/\sqrt{2}$ so that its variance also equals 1.

For **each** sample compute and print

-   the sample mean,
-   the *unbiased* sample variance,
-   the sample skewness, and
-   the excess kurtosis (kurtosis minus 3).

Plot both histograms (30 bins) in the same figure to visualise the
different tail behaviour.

Use the skeleton below – fill in the `<TODO>` parts.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from statsmodels.formula.api import ols
plt.close('all')  # fresh state

# Exercise 1 -- Sampling & higher‑order moments
rng = np.random.default_rng(0)
normal_sample  = rng.normal(0, 1, 2000)
laplace_sample = rng.laplace(0, 1/np.sqrt(2), 2000)

def describe(name, data):
    mean  = np.mean(data)
    var   = np.var(data, ddof=1)  # unbiased variance
    skew  = st.skew(data)
    kurt  = st.kurtosis(data, fisher=True)  # excess kurtosis
    print(f"{name:14s} | mean={mean:6.3f}  var={var:6.3f}  skew={skew:6.3f}  kurt={kurt:6.3f}")

describe("normal",  normal_sample)
describe("laplace", laplace_sample)

fig, ax = plt.subplots()
ax.hist(normal_sample, bins=30, alpha=0.7, label='Normal', density=True)
ax.hist(laplace_sample, bins=30, alpha=0.7, label='Laplace', density=True)
ax.set_xlabel("value")
ax.set_ylabel("frequency")
ax.legend()
fig.tight_layout()
plt.show()


## Exercise 2 – *Classical hypothesis tests*

Simulate two independent groups **A** and **B** of size 50 each from
N(0,1) and a *paired* “before/after” sample of size 40 from N(mu=0,
sigma=2).

1.  Perform an **unpaired** two‑sample *t*‑test to check if groups A
    and B differ in mean (use `equal_var=False`).
2.  Perform a **paired** *t*‑test to check if the mean changes between
    the before/after measurements.
3.  Draw 200 observations from a Laplace(0, 1/√2) distribution and use a
    **goodness‑of‑fit** test (`st.goodness_of_fit`) to verify whether
    the data are compatible with the true distribution.

Skeleton:

In [ ]:
# Exercise 2 -- Classical hypothesis tests
rng = np.random.default_rng(123)
group_A = rng.normal(0, 1, 50)
group_B = rng.normal(0, 1, 50)
before  = rng.normal(0, 2, 40)
after   = before + rng.normal(0, 0.5, 40)  # Add small shift

# 1) unpaired t-test
t_stat_ab, pval_ab = st.ttest_ind(group_A, group_B, equal_var=False)
print(f"Unpaired t-test A vs B: p = {pval_ab:.3g}")

# 2) paired t-test
t_stat_pair, pval_pair = st.ttest_rel(before, after)
print(f"Paired t-test before vs after: p = {pval_pair:.3g}")

# 3) Godness-of-fit for Laplace(0, 1/√2)
laplace_sample = rng.laplace(0, 1/np.sqrt(2), 200)
gof_res = st.goodness_of_fit(st.laplace, laplace_sample, known_params={'loc': 0, 'scale': 1/np.sqrt(2)})
print(f"Goodness-of-fit Laplace(0,1/√2): p = {gof_res.pvalue:.3g}")

In [ ]:
# Exercise 2 -- your code here
# rng = np.random.default_rng(123)
# group_A = <TODO>
# group_B = <TODO>
# before  = <TODO>
# after   = before + <TODO>  # Add small shift
#
# # 1) unpaired t-test
# t_stat_ab, pval_ab = <TODO>
# print(f"Unpaired t-test A vs B: p = {pval_ab:.3g}")
#
# # 2) paired t-test
# t_stat_pair, pval_pair = <TODO>
# print(f"Paired t-test before vs after: p = {pval_pair:.3g}")
#
# # 3) Godness-of-fit for Laplace(0, 1/√2)
# laplace_sample = <TODO>
# gof_res = <TODO>
# print(f"Goodness-of-fit Laplace(0,1/√2): p = {gof_res.pvalue:.3g}")

## Exercise 3 – *Bootstrap CI & Monte‑Carlo test*

Using the two independent groups *A* and *B* from exercise 2
(re‑generate them here with different parameters):

1.  Compute a 95 % **bootstrap** confidence interval for the difference
    of means (mu_A - mu_B) using `st.bootstrap` with
    `n_resamples=10_000` and the default BCa method.
2.  Using `st.monte_carlo_test`, test the null‑hypothesis that the mean
    of group A equals the mean of group B. Use 10 000 Monte‑Carlo
    resamples.

In [ ]:
# Exercise 3 -- your code here
# rng = np.random.default_rng(321)
# group_A = rng.normal(loc=0, scale=1, size=60)
# group_B = rng.normal(loc=0.3, scale=1.1, size=60)
#
# # 1) bootstrap CI for mean difference
# def mean_diff(x, y, axis):
    # """Difference of means along given axis."""
    # return x.mean(axis=axis) - y.mean(axis=axis)
# res = <TODO>
# ci_low, ci_high = res.confidence_interval
# print(f"95% bootstrap CI for μ_A-μ_B: [{ci_low:.3f}, {ci_high:.3f}]")
#
# # 2) Monte‑Carlo test for mean difference
# mc_res = <TODO>
# print(f"Monte‑Carlo test: p = {mc_res.pvalue:.3g}")

In [ ]:
# Exercise 3 -- Bootstrap CI & Monte‑Carlo test
rng = np.random.default_rng(321)
group_A = rng.normal(loc=0, scale=1, size=60)
group_B = rng.normal(loc=0.3, scale=1.1, size=60)

# 1) bootstrap CI for mean difference
def mean_diff(x, y, axis):
    """Difference of means along given axis."""
    return x.mean(axis=axis) - y.mean(axis=axis)

res = st.bootstrap((group_A, group_B), mean_diff, n_resamples=10_000,
                   confidence_level=0.95, method='BCa', random_state=rng)
ci_low, ci_high = res.confidence_interval
print(f"95% bootstrap CI for μ_A-μ_B: [{ci_low:.3f}, {ci_high:.3f}]")

# 2) Monte‑Carlo test for mean difference
def statistic(x, y):
    return np.mean(x) - np.mean(y)

mc_res = st.monte_carlo_test((group_A, group_B), statistic,
                            n_resamples=10_000, random_state=rng)
print(f"Monte‑Carlo test: p = {mc_res.pvalue:.3g}")


## Exercise 4 – *Linear model with `statsmodels`*

Load the classic **iris** data set (`sklearn.datasets.load_iris`) into a
*pandas* `DataFrame`. The target variable `species` is categorical with
levels *setosa*, *versicolor* and *virginica*.

1.  Fit an **OLS** model that explains `sepal_width` by `petal_length`
    and the categorical factor `species` (include an intercept).
2.  Print the model summary and extract the *p*-value associated with
    the coefficient of `petal_length`.
3.  Formulate and perform an **F‑test** for the null‑hypothesis that the
    coefficients of *versicolor* and *virginica* are equal.

In [ ]:
# Exercise 4 -- your code here
# from sklearn.datasets import load_iris
# iris = load_iris(as_frame=True)
# df = iris.frame  # features + target
# df.rename(columns={
#     "target": "species",
#     "sepal width (cm)": "sepal_width",
#     "petal length (cm)": "petal_length"
# }, inplace=True)
# df["species"] = df["species"].replace({0: "setosa", 1: "versicolor", 2: "virginica"})
#
# model = <TODO>
# print(<TODO>)  # summary
# p_petal = <TODO>
# print(f"p‑value for petal_length coef: {p_petal:.3g}")
#
# f_test_res = <TODO>
# print(f"F‑test versicolor = virginica: p = {float(f_test_res.pvalue):.3g}")

In [ ]:
# Exercise 4 -- Linear model with statsmodels
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
df = iris.frame  # features + target
df.rename(columns={
    "target": "species",
    "sepal width (cm)": "sepal_width",
    "petal length (cm)": "petal_length"
}, inplace=True)
df["species"] = df["species"].replace({0: "setosa", 1: "versicolor", 2: "virginica"})

model = ols('sepal_width ~ petal_length + C(species)', data=df).fit()
print(model.summary())  # summary
p_petal = model.pvalues['petal_length']
print(f"p‑value for petal_length coef: {p_petal:.3g}")

f_test_res = model.f_test('C(species)[T.versicolor] = C(species)[T.virginica]')
print(f"F‑test versicolor = virginica: p = {float(f_test_res.pvalue):.3g}")